In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# going to the repository of this note book
%cd drive/My Drive/Thesis/CPNN_PiggyBack
!ls

/content/drive/My Drive/Thesis/CPNN_PiggyBack
data		     iris-model.pickle	performance	   requirements.txt
datasets	     lab		push_to_git.ipynb  run_test
iris-model-full.pth  models		README.md	   Untitled0.ipynb


In [3]:
# IMPORT
import torch
import os
# this module is for training and functions related to data preparation
from models.training import *
# this is for importing pretrain models.
from models.pretrain import *

#from models.cpnn_others import cPNNExp
#from models.cpnn_seq import cPNNSeq
#from models.cgru_double import cGRULinearDouble
#from models.clstm import *

import torch.nn.functional as F
import numpy as np
import pandas as pd
import pickle
import argparse
from torch.autograd import Variable
import pprint
import copy


In [4]:
# EDITABLE PARAMETERS
dataset = "sine_rw10_mode5_extended_2-1_1234"

# OTHER PARAMETERS
batch_size = 128
hidden_size = 50
seq_len = 10
# TODO
iterations = 1
loss_on_seq = False
freeze_inputs_weights = False
pretraining_samples = 0
pretraining_epochs = 0
write_weights = False
combination = False
rembember_initial_states = False
model_type='GRU'
if model_type == "GRU":
    model_class = PretrainModel
else:
    model_class = cGRULinearDouble

path = os.path.join(
    "performance/pretrain_GRU"
)

In [5]:
df = pd.read_csv("datasets/datasets/sine_rw10_mode5_extended_2-1_1234.csv")
df_task = df[df["task"] == 1]
df_task = df_task.drop(columns="task")

In [ ]:
Model = PretrainModel(input_size=2,
        device=torch.device("cpu"),
      	num_layers=1,
        hidden_size=50,
        output_size=2,
        batch_size=128,
          )
instance = cPNN(seq_len=seq_len)

In [ ]:
if len(df_task) % batch_size == 0:
  n_batches = int(len(df_task) / batch_size)
else:
  n_batches = int(len(df_task) / batch_size) + 1
optimizer = torch.optim.Adam(Model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss(reduction="mean")
for j in range(0,3):
  for i in range(0, len(df_task), batch_size):
    x = df_task.iloc[i : i + batch_size, 0:-1].values.astype(np.float32)
    y = list(df_task.iloc[i : i + batch_size, -1])
    if len(y) >= seq_len:
      x = np.array(x)
      y = list(y)
      x, y, _ = instance._load_batch(x, y)
      y_pred = Model(x)

      y_pred = get_samples_outputs(y_pred)
      pred, _ = get_pred_from_outputs(y_pred)
      loss = loss_fn(y_pred, y)
      # backward pass
      optimizer.zero_grad()
      loss.backward()
      # update weights
      optimizer.step()


In [ ]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(Model.state_dict())

In [ ]:
# Save model
with open("performance/pretrain/pretrain_mode_gru.pickle", "wb") as fp:
    pickle.dump(Model.state_dict(), fp)

## trying to read the weights of the pretrain model

In [ ]:
pretrain_model = PretrainModel(input_size=2,
        device=torch.device("cpu"),
      	num_layers=1,
        hidden_size=50,
        output_size=2,
        batch_size=128,
          )
with open("performance/pretrain/pretrain_mode_gru.pickle", "rb") as fp:
    pretrain_model.load_state_dict(pickle.load(fp),strict=False)

In [ ]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(pretrain_model.state_dict())

OrderedDict([   (   'gru.weight_ih_l0',
                    tensor([[ 4.9395e-01,  7.2624e-01],
        [-2.5023e-01, -1.5779e+00],
        [ 4.2910e-02, -2.2821e-03],
        [-2.7951e-01, -9.4084e-02],
        [-4.2295e-01, -2.2655e-01],
        [-7.1322e-01, -6.1827e-01],
        [-1.0246e-02, -3.2005e-01],
        [-4.1410e-01, -1.2440e+00],
        [-7.2617e-01, -7.9609e-01],
        [-5.6529e-01, -9.9232e-01],
        [-5.3829e-01, -1.3724e+00],
        [-4.4110e-01, -1.1477e+00],
        [-4.6161e-01, -9.9819e-01],
        [-3.8753e-01,  8.9283e-01],
        [-6.3671e-01, -1.5068e+00],
        [-7.4016e-01, -1.4635e+00],
        [-2.6262e-01, -5.4108e-02],
        [-8.1440e-01, -1.1812e+00],
        [-2.4015e-01, -5.9275e-01],
        [-7.2938e-01, -1.5080e+00],
        [ 1.7937e-01,  2.7415e-01],
        [-4.2030e-01, -5.6660e-01],
        [-5.5590e-01, -5.9604e-01],
        [-5.3422e-01, -5.8739e-01],
        [-3.8810e-01,  2.8452e-01],
        [-9.4800e-02,  7.1795e-01],
    

In [ ]:
pretrain_model

PretrainModel(
  (gru): GRU(2, 50, batch_first=True)
  (linear): Linear(in_features=50, out_features=2, bias=True)
)

In [ ]:
x = copy.deepcopy(pretrain_model.state_dict())
for param_name in x:
  print(param_name)
  print(x[param_name].shape)

gru.weight_ih_l0
torch.Size([150, 2])
gru.weight_hh_l0
torch.Size([150, 50])
gru.bias_ih_l0
torch.Size([150])
gru.bias_hh_l0
torch.Size([150])
linear.weight
torch.Size([2, 50])
linear.bias
torch.Size([2])


#**From now on, the part for adding mask will be implemented:**

In [6]:
batch_size = 128
hidden_size = 50
seq_len = 10
mask_init='uniform'
dataset = "sine_rw10_mode5_extended_15-10_1234"
df = pd.read_csv(os.path.join("datasets/datasets", f"{dataset}.csv"))
pretrain_model_addr='performance/pretrain/pretrain_mode_gru.pickle'
mask_weights=[]
number_of_tasks=4
model=cPNN(model_class=PiggyBackGRU, lr = 0.001, seq_len=seq_len, pretrain_model_addr=pretrain_model_addr,
           mask_weights=mask_weights, mask_init=mask_init, number_of_tasks=number_of_tasks)
iteration=1
inner_iteration=15
# first loop for the iterations, for now it is just 1 iteration.
temporal_mask_saving=[]
acc_saving=[]
cohen_kappa_saving=[]
all_batch_acc=[]
all_batch_kappa=[]
for iter in range(0,iteration):
  mask_weights.append([])


  # loop for each task
  for task in range(1, df["task"].max() + 1):

    all_batch_acc=0
    all_batch_kappa=0
    mask_weights[-1].append([])
    temporal_mask_saving= copy.deepcopy(mask_weights)
    acc_saving=np.zeros(task)
    cohen_kappa_saving=np.zeros(task)
    df_task = df[df["task"] == task]
    df_task = df_task.drop(columns="task")
    # loop based on each batch of data
    batch_cont=0
    for i in range(0, len(df_task), batch_size):
      x = df_task.iloc[i : i + batch_size, 0:-1].values.astype(np.float32)
      y = list(df_task.iloc[i : i + batch_size, -1])
      if batch_cont<100:
        for mask_index in range (0, task):
          model_weights = copy.deepcopy(temporal_mask_saving[-1][mask_index])
          for j in range(0,inner_iteration):
            model_weights,acc, cohen_kappa_value =model.learn_many(x,y,model_weights)
            temporal_mask_saving[-1][mask_index] = copy.deepcopy(model_weights)
            acc_saving[mask_index]+=acc
            cohen_kappa_saving[mask_index]+=cohen_kappa_value
        batch_cont+=1
      elif batch_cont==100:
        avg_acc= [x / (batch_cont*inner_iteration) for x in acc_saving]
        avg_cohen_kappa = [x / (batch_cont*inner_iteration) for x in cohen_kappa_saving]
        print(avg_acc)
        print(avg_cohen_kappa)
        index_of_best_acc = np.argmax(avg_acc)
        print('index',index_of_best_acc)
        model_weights = copy.deepcopy(temporal_mask_saving[-1][index_of_best_acc])
        all_batch_acc=avg_acc[index_of_best_acc] * (batch_cont*inner_iteration)
        all_batch_kappa=avg_cohen_kappa[index_of_best_acc] * (batch_cont*inner_iteration)
        for j in range(0,inner_iteration):
          model_weights,acc,cohen_kappa_value= model.learn_many(x,y,model_weights)
          all_batch_acc+=acc
          all_batch_kappa+=cohen_kappa_value

        batch_cont+=1
      elif batch_cont>100:
        batch_cont+=1
        for j in range(0,inner_iteration):
          model_weights,acc,cohen_kappa_value= model.learn_many(x,y,model_weights)
          all_batch_acc+=acc
          all_batch_kappa+=cohen_kappa_value

    #print(model_weights)
    print('all batches Accuracy= ', all_batch_acc / (batch_cont*inner_iteration))
    print('All batches cohen kappa= ', all_batch_kappa / (batch_cont*inner_iteration))
    mask_weights[-1][-1]=copy.deepcopy(model_weights)


[0.8868854166666666]
[0.7703007828742265]
index 0
all batches Accuracy=  0.9117356670929244
All batches cohen kappa=  0.8203634322007193
[0.9113020833333333, 0.9193020833333333]
[0.813602383269618, 0.8285903065999349]
index 1
all batches Accuracy=  0.9433280051150893
All batches cohen kappa=  0.8800325899681065
[0.9310572916666666, 0.849640625, 0.852765625]
[0.8589825894633929, 0.700902953290691, 0.7048970191770544]
index 0
all batches Accuracy=  0.9319416027280479
All batches cohen kappa=  0.8611378891439593
[0.9113802083333333, 0.9564270833333334, 0.9057447916666667, 0.9114583333333334]
[0.814698654666543, 0.9076823399464289, 0.8022406987051169, 0.8139188839594523]
index 1
all batches Accuracy=  0.9555602621483373
All batches cohen kappa=  0.9058041118193364


In [7]:
batch_size = 128
hidden_size = 50
seq_len = 10
mask_init='uniform'
dataset = "sine_rw10_mode5_extended_16-8_1234"
df = pd.read_csv(os.path.join("datasets/datasets", f"{dataset}.csv"))
pretrain_model_addr='performance/pretrain/pretrain_mode_gru.pickle'
mask_weights=[]
number_of_tasks=4
model=cPNN(model_class=PiggyBackGRU, lr = 0.001, seq_len=seq_len, pretrain_model_addr=pretrain_model_addr,
           mask_weights=mask_weights, mask_init=mask_init, number_of_tasks=number_of_tasks)
iteration=1
inner_iteration=15
# first loop for the iterations, for now it is just 1 iteration.
temporal_mask_saving=[]
acc_saving=[]
cohen_kappa_saving=[]
all_batch_acc=[]
all_batch_kappa=[]
for iter in range(0,iteration):
  mask_weights.append([])


  # loop for each task
  for task in range(1, df["task"].max() + 1):

    all_batch_acc=0
    all_batch_kappa=0
    mask_weights[-1].append([])
    temporal_mask_saving= copy.deepcopy(mask_weights)
    acc_saving=np.zeros(task)
    cohen_kappa_saving=np.zeros(task)
    df_task = df[df["task"] == task]
    df_task = df_task.drop(columns="task")
    # loop based on each batch of data
    batch_cont=0
    for i in range(0, len(df_task), batch_size):
      x = df_task.iloc[i : i + batch_size, 0:-1].values.astype(np.float32)
      y = list(df_task.iloc[i : i + batch_size, -1])
      if batch_cont<100:
        for mask_index in range (0, task):
          model_weights = copy.deepcopy(temporal_mask_saving[-1][mask_index])
          for j in range(0,inner_iteration):
            model_weights,acc, cohen_kappa_value =model.learn_many(x,y,model_weights)
            temporal_mask_saving[-1][mask_index] = copy.deepcopy(model_weights)
            acc_saving[mask_index]+=acc
            cohen_kappa_saving[mask_index]+=cohen_kappa_value
        batch_cont+=1
      elif batch_cont==100:
        avg_acc= [x / (batch_cont*inner_iteration) for x in acc_saving]
        avg_cohen_kappa = [x / (batch_cont*inner_iteration) for x in cohen_kappa_saving]
        print(avg_acc)
        print(avg_cohen_kappa)
        index_of_best_acc = np.argmax(avg_acc)
        print('index',index_of_best_acc)
        model_weights = copy.deepcopy(temporal_mask_saving[-1][index_of_best_acc])
        all_batch_acc=avg_acc[index_of_best_acc] * (batch_cont*inner_iteration)
        all_batch_kappa=avg_cohen_kappa[index_of_best_acc] * (batch_cont*inner_iteration)
        for j in range(0,inner_iteration):
          model_weights,acc,cohen_kappa_value= model.learn_many(x,y,model_weights)
          all_batch_acc+=acc
          all_batch_kappa+=cohen_kappa_value

        batch_cont+=1
      elif batch_cont>100:
        batch_cont+=1
        for j in range(0,inner_iteration):
          model_weights,acc,cohen_kappa_value= model.learn_many(x,y,model_weights)
          all_batch_acc+=acc
          all_batch_kappa+=cohen_kappa_value

    #print(model_weights)
    print('all batches Accuracy= ', all_batch_acc / (batch_cont*inner_iteration))
    print('All batches cohen kappa= ', all_batch_kappa / (batch_cont*inner_iteration))
    mask_weights[-1][-1]=copy.deepcopy(model_weights)


[0.87096875]
[0.7373046035797646]
index 0
all batches Accuracy=  0.9008530477408359
All batches cohen kappa=  0.7985320263717701
[0.9229947916666666, 0.921453125]
[0.8389717309375604, 0.8352876234551271]
index 0
all batches Accuracy=  0.9435563192668371
All batches cohen kappa=  0.8810109921199891
[0.9120989583333333, 0.77978125, 0.823328125]
[0.8209014601508776, 0.5545785898912388, 0.6398793853993218]
index 0
all batches Accuracy=  0.919386455669224
All batches cohen kappa=  0.8361007302313509
[0.911171875, 0.95228125, 0.8961770833333333, 0.9084427083333333]
[0.8167445481059452, 0.8981419191757838, 0.7846443550263842, 0.806660264412562]
index 1
all batches Accuracy=  0.9469019075021314
All batches cohen kappa=  0.887484134381791
